<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/test_gcn_variants.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!python -c "import torch; print(torch.__version__)"


2.6.0+cu124


In [3]:
!python -c "import torch; print(torch.version.cuda)"



12.4


In [4]:
pip install torchvision

In [5]:
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html



Looking in links: https://data.pyg.org/whl/torch-2.6.0+cu124.html


In [19]:
# Standard libraries
import numpy as np
from scipy import sparse
import seaborn as sns
import pandas as pd
import time

# Plotting libraries
import matplotlib.pyplot as plt
import networkx as nx
from matplotlib import cm
from IPython.display import Javascript  # Restrict height of output cell.

# PyTorch
import torch
import torch.nn.functional as F
from torch.nn import Linear
import torch.nn as nn
# import pyg_lib
import torch_sparse
from sklearn.metrics import f1_score

# PyTorch geometric
from torch_geometric.nn import GCNConv
from torch_geometric.datasets import Planetoid, Amazon
from torch_geometric.loader import ClusterData, ClusterLoader
from torch_geometric.transforms import NormalizeFeatures
from torch_geometric.data import Data
from torch_geometric import seed_everything
from torch_geometric.nn.models import GraphSAGE
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit
import torch_geometric.transforms as T
import json


**STANDARD GCN pubmed**

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Planetoid(root='data/Planetoid', name='PubMed', transform=NormalizeFeatures())
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data

Using device: cuda


Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [8]:
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')

Number of nodes:          19717
Number of edges:          88648
Average node degree:      4.50
Number of training nodes: 60
Training node label rate: 0.003
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True


In [9]:
def clean_gpu_memory():
    """Cleans GPU memory without fully resetting the CUDA context"""
    import gc
    gc.collect()  # Python garbage collection
    if torch.cuda.is_available():
        torch.cuda.empty_cache()  # PyTorch cache
        torch.cuda.reset_peak_memory_stats()  # Reset tracking
        print(f"Memory after cleanup: {torch.cuda.memory_allocated()/1024**2:.2f} MB")


In [10]:
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, dropout=0.5):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.dropout = dropout

    def forward(self, x, edge_index, edge_weight=None):
        x = self.conv1(x, edge_index, edge_weight).relu()
        x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.conv2(x, edge_index, edge_weight)
        return F.log_softmax(x, dim=-1)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

print(model)

GCN(
  (conv1): GCNConv(500, 64)
  (conv2): GCNConv(64, 3)
)


In [12]:
def train(data, mask):
  model.train()
  optimizer.zero_grad()  # Clear gradients.
  out = model(data.x, data.edge_index)  # Perform a single forward pass.
  loss = criterion(out[mask], data.y[mask])  # Compute the loss solely based on the training nodes.
  loss.backward()  # Derive gradients.
  optimizer.step()  # Update parameters based on gradients.
  return loss

def test(data, mask):
    model.eval()
    out = model(data.x, data.edge_index)
    pred = out.argmax(dim=1)
    true = data.y[mask].cpu()
    pred = pred[mask].cpu()
    acc = (pred == true).sum().item() / mask.sum().item()
    micro_f1 = f1_score(true, pred, average='micro')
    return acc, micro_f1


In [13]:

#display(Javascript('''google.colab.output.setIframeHeight(0, true, {maxHeight: 300})'''))
start_time = time.time()

for epoch in range(1, 101):
    loss = train(data, data.train_mask)
    train_acc, train_f1 = test(data, data.train_mask)
    val_acc, val_f1 = test(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')


end_time = time.time()
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
print(f"Time taken: {end_time - start_time:.2f} seconds")

Epoch: 001, Train Acc: 0.6167, Train F1: 0.6167, Val Acc: 0.5360, Val F1: 0.5360
Epoch: 002, Train Acc: 0.8500, Train F1: 0.8500, Val Acc: 0.7060, Val F1: 0.7060
Epoch: 003, Train Acc: 0.8333, Train F1: 0.8333, Val Acc: 0.6680, Val F1: 0.6680
Epoch: 004, Train Acc: 0.8500, Train F1: 0.8500, Val Acc: 0.6680, Val F1: 0.6680
Epoch: 005, Train Acc: 0.8667, Train F1: 0.8667, Val Acc: 0.6720, Val F1: 0.6720
Epoch: 006, Train Acc: 0.9000, Train F1: 0.9000, Val Acc: 0.7020, Val F1: 0.7020
Epoch: 007, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7140, Val F1: 0.7140
Epoch: 008, Train Acc: 0.9333, Train F1: 0.9333, Val Acc: 0.7160, Val F1: 0.7160
Epoch: 009, Train Acc: 0.9333, Train F1: 0.9333, Val Acc: 0.7300, Val F1: 0.7300
Epoch: 010, Train Acc: 0.9167, Train F1: 0.9167, Val Acc: 0.7320, Val F1: 0.7320
Epoch: 011, Train Acc: 0.9500, Train F1: 0.9500, Val Acc: 0.7380, Val F1: 0.7380
Epoch: 012, Train Acc: 0.9500, Train F1: 0.9500, Val Acc: 0.7380, Val F1: 0.7380
Epoch: 013, Train Acc: 0.950

In [14]:
test_acc,f1_micro = test(data, data.test_mask)
test_acc

0.765

In [15]:
clean_gpu_memory()

Memory after cleanup: 56.31 MB


In [16]:
def gcn_memory_usage(num_layers: int, num_nodes: int, feat_dim: int, use_bytes: bool = True):

    # Each embedding matrix is size |V| × K floats, there are L such layers
    num_emb_floats = num_layers * num_nodes * feat_dim
    # Each weight matrix is K × K floats per layer
    num_weight_floats = num_layers * feat_dim * feat_dim

    if use_bytes:
        bytes_per_float = 4  # assuming float32
        embeddings_bytes = num_emb_floats * bytes_per_float
        weights_bytes = num_weight_floats * bytes_per_float
        total_bytes = embeddings_bytes + weights_bytes
        return embeddings_bytes, weights_bytes, total_bytes
    else:
        return num_emb_floats, num_weight_floats, num_emb_floats + num_weight_floats


e, w, t = gcn_memory_usage(2, data.num_nodes, data.num_features)
print(f"Embeddings: {e/1e6:.1f} MB, Weights: {w*4/1e6:.1f} MB, Total: {t/1e6:.1f} MB")
peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "GCN full-batch",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "embedding_storage":e,
    "Weight_Matrices":w,
    "Total_Memory":t
}

with open("GCN_full_batch_pubmed_results.json", "w") as f:
    json.dump(metrics, f)

Embeddings: 78.9 MB, Weights: 8.0 MB, Total: 80.9 MB


**Cora dataset**

In [17]:
clean_gpu_memory()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Planetoid(root='data/Planetoid', name='Cora', transform=NormalizeFeatures())
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

start_time = time.time()

for epoch in range(1, 101):
    loss = train(data, data.train_mask)
    train_acc, train_f1 = test(data, data.train_mask)
    val_acc, val_f1 = test(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')


end_time = time.time()
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
print(f"Time taken: {end_time - start_time:.2f} seconds")

test_acc,f1_micro = test(data, data.test_mask)
test_acc

e, w, t = gcn_memory_usage(2, data.num_nodes, data.num_features)
print(f"Embeddings: {e/1e6:.1f} MB, Weights: {w*4/1e6:.1f} MB, Total: {t/1e6:.1f} MB")


peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "GCN full-batch",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "embedding_storage":e,
    "Weight_Matrices":w,
    "Total_Memory":t
}

with open("GCN_full_batch_cora_results.json", "w") as f:
    json.dump(metrics, f)

Memory after cleanup: 56.31 MB
Using device: cuda
Number of nodes:          2708
Number of edges:          10556
Average node degree:      3.90
Number of training nodes: 140
Training node label rate: 0.052
Has isolated nodes:       False
Has self-loops:           False
Is undirected:            True
Epoch: 001, Train Acc: 0.6857, Train F1: 0.6857, Val Acc: 0.4140, Val F1: 0.4140
Epoch: 002, Train Acc: 0.8500, Train F1: 0.8500, Val Acc: 0.5680, Val F1: 0.5680
Epoch: 003, Train Acc: 0.9214, Train F1: 0.9214, Val Acc: 0.6600, Val F1: 0.6600
Epoch: 004, Train Acc: 0.9643, Train F1: 0.9643, Val Acc: 0.7040, Val F1: 0.7040
Epoch: 005, Train Acc: 0.9786, Train F1: 0.9786, Val Acc: 0.7580, Val F1: 0.7580
Epoch: 006, Train Acc: 0.9786, Train F1: 0.9786, Val Acc: 0.7480, Val F1: 0.7480
Epoch: 007, Train Acc: 0.9857, Train F1: 0.9857, Val Acc: 0.7500, Val F1: 0.7500
Epoch: 008, Train Acc: 0.9857, Train F1: 0.9857, Val Acc: 0.7420, Val F1: 0.7420
Epoch: 009, Train Acc: 0.9857, Train F1: 0.9857, Va

**Citeseer dataset**

In [18]:
clean_gpu_memory()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Planetoid(root='data/Planetoid', name='CiteSeer', transform=NormalizeFeatures())
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

start_time = time.time()

for epoch in range(1, 101):
    loss = train(data, data.train_mask)
    train_acc, train_f1 = test(data, data.train_mask)
    val_acc, val_f1 = test(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')


end_time = time.time()
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
print(f"Time taken: {end_time - start_time:.2f} seconds")

test_acc,f1_micro = test(data, data.test_mask)
test_acc

e, w, t = gcn_memory_usage(2, data.num_nodes, data.num_features)
print(f"Embeddings: {e/1e6:.1f} MB, Weights: {w*4/1e6:.1f} MB, Total: {t/1e6:.1f} MB")


peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "GCN full-batch",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "embedding_storage":e,
    "Weight_Matrices":w,
    "Total_Memory":t
}

with open("GCN_full_batch_citeseer_results.json", "w") as f:
    json.dump(metrics, f)

Memory after cleanup: 72.22 MB
Using device: cuda
Number of nodes:          3327
Number of edges:          9104
Average node degree:      2.74
Number of training nodes: 120
Training node label rate: 0.036
Has isolated nodes:       True
Has self-loops:           False
Is undirected:            True
Epoch: 001, Train Acc: 0.1917, Train F1: 0.1917, Val Acc: 0.0680, Val F1: 0.0680
Epoch: 002, Train Acc: 0.4333, Train F1: 0.4333, Val Acc: 0.1060, Val F1: 0.1060
Epoch: 003, Train Acc: 0.8250, Train F1: 0.8250, Val Acc: 0.3680, Val F1: 0.3680
Epoch: 004, Train Acc: 0.8917, Train F1: 0.8917, Val Acc: 0.5060, Val F1: 0.5060
Epoch: 005, Train Acc: 0.9500, Train F1: 0.9500, Val Acc: 0.6100, Val F1: 0.6100
Epoch: 006, Train Acc: 0.9583, Train F1: 0.9583, Val Acc: 0.6640, Val F1: 0.6640
Epoch: 007, Train Acc: 0.9667, Train F1: 0.9667, Val Acc: 0.6440, Val F1: 0.6440
Epoch: 008, Train Acc: 0.9667, Train F1: 0.9667, Val Acc: 0.6420, Val F1: 0.6420
Epoch: 009, Train Acc: 0.9667, Train F1: 0.9667, Val 

**Amazon dataset**

In [20]:
clean_gpu_memory()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
dataset = Amazon(
        root='data/Amazon',
        name='Computers',
        transform=T.Compose([
        NormalizeFeatures(),          # feature‑wise ℓ₂ normalisation
        RandomNodeSplit(              # ⇦ add a split transform
                split='train_rest',       # 10% val, 10% test by default
                num_val=0.1,
                num_test=0.1,
                num_splits=1,
            )
        ])
    )
num_features = dataset.num_features
num_classes = dataset.num_classes
data = dataset[0].to(device)  # Get the first graph object.
data
print(f'Number of nodes:          {data.num_nodes}')
print(f'Number of edges:          {data.num_edges}')
print(f'Average node degree:      {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.3f}')
print(f'Has isolated nodes:       {data.has_isolated_nodes()}')
print(f'Has self-loops:           {data.has_self_loops()}')
print(f'Is undirected:            {data.is_undirected()}')



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(num_features, 64, num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

start_time = time.time()

for epoch in range(1, 101):
    loss = train(data, data.train_mask)
    train_acc, train_f1 = test(data, data.train_mask)
    val_acc, val_f1 = test(data, data.val_mask)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Train F1: {train_f1:.4f}, Val Acc: {val_acc:.4f}, Val F1: {val_f1:.4f}')


end_time = time.time()
print(f"Current GPU memory: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
print(f"Max GPU memory used: {torch.cuda.max_memory_allocated()/1024**2:.2f} MB")
print(f"Time taken: {end_time - start_time:.2f} seconds")

test_acc,f1_micro = test(data, data.test_mask)
test_acc

e, w, t = gcn_memory_usage(2, data.num_nodes, data.num_features)
print(f"Embeddings: {e/1e6:.1f} MB, Weights: {w*4/1e6:.1f} MB, Total: {t/1e6:.1f} MB")


peak_memory_mb=f"{torch.cuda.max_memory_allocated()/1024**2:.2f}"
total_train_time=f"{end_time - start_time:.2f}"

metrics = {
    "model": "GCN full-batch",
    "accuracy": test_acc,
    "f1_micro":f1_micro,
    "peak_memory_MB": peak_memory_mb,
    "train_time_sec": total_train_time,
    "embedding_storage":e,
    "Weight_Matrices":w,
    "Total_Memory":t
}

with open("GCN_full_batch_Amazon_results.json", "w") as f:
    json.dump(metrics, f)

Memory after cleanup: 106.61 MB
Using device: cuda


Processing...
Done!


Number of nodes:          13752
Number of edges:          491722
Average node degree:      35.76
Number of training nodes: 11002
Training node label rate: 0.800
Has isolated nodes:       True
Has self-loops:           False
Is undirected:            True
Epoch: 001, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 002, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 003, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 004, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 005, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 006, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 007, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 008, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 009, Train Acc: 0.3761, Train F1: 0.3761, Val Acc: 0.3847, Val F1: 0.3847
Epoch: 010, Trai